# Leitura da Base

In [1]:
import pandas as pd
import re
import os
import joblib
import collections
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy
import nltk
from rouge_score import rouge_scorer
from collections import Counter
import fasttext
import numpy as np

In [2]:
if os.path.exists('lista_papers'):
    lista_papers = joblib.load('lista_papers')
else:
    list_of_files = os.listdir('scisummnet_release1.1__20190413/top1000_complete')

    list_of_files2= []
    for i in list_of_files:
        nome = 'scisummnet_release1.1__20190413/top1000_complete/' + i + '/' + 'Documents_xml/' +os.listdir(f'scisummnet_release1.1__20190413/top1000_complete/{i}/Documents_xml')[0]
        list_of_files2.append(nome)


    lines = []
    for file in list_of_files2:
        f= open(file,'r')
        lines.append(f.readlines())
        f.close()

    lista_papers=[]
    for j in lines:
        texto=''
        for i in j:
            texto = texto + str(i)
        lista_papers.append(texto)

        joblib.dump(lista_papers, 'lista_papers')

In [3]:
len(lista_papers)

1009

In [4]:
#Aplicando indices aos papers
indices           = range(0,len(lista_papers))
papers_com_indice = dict(zip(indices, lista_papers))

# Estudo da Base

In [5]:
import string

In [6]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<SECTION.*[Ii][nN][tT][rR][oO][dD][Uu][Cc][Tt][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub('&amp;quot;', '', txt)
        txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        lista.append(txt)
    return txt

def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<[Aa][Bb][sS][tT][Rr][aA][cC][tT]>)(.+?)(</[aA][bB][sS][tT][rR][aA][cC][tT]>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub('&amp;quot;', '', txt)
        txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub(r'^(\d)*(\.)?([0-9]{1})?', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        lista.append(txt)
    return txt


def preprocessa_conclusion(corpus: str):
    regIter1 = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos1 = [ t.group(2) for t in regIter1]
    regIter2 = re.finditer('(<SECTION.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos2 = [ t.group(2) for t in regIter2]
    regIter3 = re.finditer('(<SECTION.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Ee][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos3 = [ t.group(2) for t in regIter3]
    textos = textos1
    for i in textos2:
        if i not in textos:
            textos.append(i)
    for i in textos3:
        if i not in textos:
            textos.append(i)
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub('&amp;quot;', '', txt)
        txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
    return txt

In [8]:
def get_indices_preprocessamento(x, funcao_preprocessamento):
    """ alterar a funcao_preprocessamento entre os parâmetros:
    preprocessa_intro, preprocessa_abstract, preprocessa_conclusion"""
    papers_processados=[]
    indices=[]
    for i,j in zip(list(x.keys()),list(x.values())):
        try:
            papers_processados.append(funcao_preprocessamento(j))
            indices.append(i)
        except:
            None
    return indices

In [9]:
indices_papers_com_abstract   = get_indices_preprocessamento(papers_com_indice, preprocessa_abstract)
indices_papers_com_introducao = get_indices_preprocessamento(papers_com_indice, preprocessa_intro)
indices_papers_com_conclusao  = get_indices_preprocessamento(papers_com_indice, preprocessa_conclusion)

In [10]:
print('qtd de papers com abstract', len(indices_papers_com_abstract))
print('qtd de papers com introdução', len(indices_papers_com_introducao))
print('qtd de papers com conclusão', len(indices_papers_com_conclusao))

print('---------------------------------')

print('qtd total de papers', len(lista_papers))

qtd de papers com abstract 924
qtd de papers com introdução 802
qtd de papers com conclusão 831
---------------------------------
qtd total de papers 1009


Contagem das sessões a partir do título faltante para melhoria do Regex

In [11]:
papers_sem_abstract = dict((key,value) for key, value in papers_com_indice.items() if key  not in  indices_papers_com_abstract)
papers_sem_introducao = dict((key,value) for key, value in papers_com_indice.items() if key  not in  indices_papers_com_introducao)
papers_sem_conclusao = dict((key,value) for key, value in papers_com_indice.items() if key  not in  indices_papers_com_conclusao)

In [12]:
def contador_sessoes(x):
    """input: dicionário de papers
       output: lista contendo o título das sessões"""
    titulos_sessoes=[]
    for i in x.values():
        regIter = re.finditer('(<SECTION(.*?)>)', i, re.DOTALL|re.MULTILINE)
        textos = [ t.group(2) for t in regIter]
        titulos_sessoes.append(textos)

    x=[]
    for i in titulos_sessoes:
        for j in i:
            pattern = r'"(.*?)"'
            matches = re.findall(pattern,str(j))
            pattern2 = r'[0-9]'
            new_string = re.sub(pattern2, '', matches[0])
            x.append(new_string.strip())

    frequency = collections.Counter(x)
    return sorted(frequency.items(), key=lambda x:x[1], reverse=True)

In [13]:
contador_sessoes(papers_sem_abstract)

[('.', 20),
 ('Introduction', 12),
 ('Acknowledgements', 6),
 ('Results', 5),
 ('', 4),
 ('Related Work', 4),
 ('Acknowledgments', 3),
 ('Abstract  Introduction', 3),
 ('Data and Evaluation', 3),
 ('Concluding Remarks', 3),
 ('Conclusion', 2),
 ('Experimental Results', 2),
 ('Conclusions and Future Work', 2),
 ('Conditional Random Fields', 2),
 ('The Joyce System in the Ulysses User Interface', 1),
 ('The Structure of Joyce', 1),
 ('The Text Planner', 1),
 ('The Sentence Planner', 1),
 ('The Linguistic Realizer', 1),
 ('An Example', 1),
 ('ENTER AGENT #&lt;information&gt; OBJECT INCOMPONENT Black Box&gt; LOCATION #&lt;PORT P&gt;',
  1),
 ('Porting the System', 1),
 ('TRANSLATION DELIVERY SYSTEM', 1),
 ('TESTING AND EVALUATING MULTI-ENGINE PERFORMANCE', 1),
 ('CURRENT AND FUTURE WORK', 1),
 ('Input Structure', 1),
 ('System Architecture', 1),
 ('Linguistic Knowledge Bases', 1),
 ('Interfaces', 1),
 ('System Performance', 1),
 ('Status', 1),
 ('. Corpus-based Methods', 1),
 ('Arabic Lang

In [14]:
contador_sessoes(papers_sem_introducao)

[('.', 549),
 ('', 83),
 ('Acknowledgments', 63),
 ('Conclusion', 15),
 ('Motivation', 13),
 ('Acknowledgements', 9),
 ('Conclusions', 9),
 ('Related Work', 8),
 ('Definition', 8),
 ('Evaluation', 7),
 ('Background', 5),
 ('Acknowledgment', 4),
 ('Experiments', 4),
 ('Results', 4),
 ('Appendix', 3),
 ('References', 3),
 ('Conclusion and Future Work', 3),
 ('Discussion', 3),
 ('ACKNOWLEDGEMENTS', 3),
 ('Conclusions and future work', 2),
 ('Overview', 2),
 ('Related Work.', 2),
 ('. Results', 2),
 ('AT&amp;T Bell Laboratories AT&amp;T Bell Laboratories', 2),
 ('Applications', 2),
 ('Approach', 2),
 ('Conclusions and Future Work', 2),
 ('Summary', 2),
 ('English', 2),
 ('Resources', 2),
 ('Discussion and related work', 1),
 ('(-. &amp;quot;NN&amp;quot;)', 1),
 ('TRANSLATION DELIVERY SYSTEM', 1),
 ('TESTING AND EVALUATING MULTI-ENGINE PERFORMANCE', 1),
 ('CURRENT AND FUTURE WORK', 1),
 ('.. The Amount of Training Data Required', 1),
 ('Proper Name Identification in Natural Language Process

In [15]:
#Results, Discussion, Concluding remarks, Experiments and Results, Conclustion and Future Research
contador_sessoes(papers_sem_conclusao)

[('.', 176),
 ('Introduction', 69),
 ('Discussion', 27),
 ('', 18),
 ('Experiments', 11),
 ('Related Work', 8),
 ('Results', 8),
 ('Acknowledgment', 7),
 ('Experimental Results', 4),
 ('Evaluation.', 3),
 ('Discussion.', 3),
 ('Summary', 3),
 ('Evaluation', 3),
 ('Acknowledgement', 3),
 ('Support Vector Machines', 3),
 ('Previous Work', 3),
 ('Motivation', 3),
 ('Experiments.', 2),
 ('Summary.', 2),
 ('Related work', 2),
 ('. Results', 2),
 ('Method', 2),
 ('REFERENCES', 2),
 ('INTRODUCTION', 2),
 ('TRANSLATION DELIVERY SYSTEM', 1),
 ('TESTING AND EVALUATING MULTI-ENGINE PERFORMANCE', 1),
 ('CURRENT AND FUTURE WORK', 1),
 ('.. The Amount of Training Data Required', 1),
 ('Grammar Transformation.', 1),
 ('LM Adaptation via Sentence Retrieval.', 1),
 ('Structured Query Models.', 1),
 ('Decision Trees.', 1),
 ('Splitting of the POS Tags.', 1),
 ('Our Tagger.', 1),
 ('Motivation and Theoretical implications.', 1),
 ('A CUG Grammar  Model that Aecomodates Word', 1),
 ('Previous work.', 1),


Identificação dos papers com as 3 sessões simultâneamente

In [16]:
indice_papers=[]
for idx in list(indices_papers_com_abstract):
    if idx in list(indices_papers_com_introducao):
        if idx in list(indices_papers_com_conclusao):
            indice_papers.append(idx)

In [17]:
print('Quantidade de papers com as três sessões simulataneamente:',len(indice_papers))

Quantidade de papers com as três sessões simulataneamente: 707


In [18]:
indices_faltantes=[]
for i in indices:
    if i not in indice_papers:
        indices_faltantes.append(i)

In [19]:
print('Quantidade de papers sem as três sessões:',len(indices_faltantes))

Quantidade de papers sem as três sessões: 302


papers_de_fora = dict((key,value) for key, value in papers_com_indice.items() if key  in indices_faltantes)

contador_sessoes(papers_de_fora)

In [20]:
papers = dict((key,value) for key, value in papers_com_indice.items() if key in indice_papers)

Seguir o estudo com os artigos armazenados na variável `papers`. \
Os armazenados na variável `papers_de_fora` não entrarão no estudo.

# Separação das Seções

In [21]:
#Definição do Abstract
abstract=[]
for i in papers.values():
    abstract.append(preprocessa_abstract(i))
    
#Definição da Introdução
intro=[]
for i in papers.values():
    intro.append(preprocessa_intro(i))
    
#Definição da Conclusão
conclusion=[]
for i in papers.values():
    conclusion.append(preprocessa_conclusion(i))
    
#Tokens
token_abstract   = []
for texto in abstract:
    token_abstract.append(texto.split())
    
token_intro   = []
for texto in intro:
    token_intro.append(texto.split())
    
token_conclusion   = []
for texto in conclusion:
    token_conclusion.append(texto.split())

tokens = token_abstract+token_intro+token_conclusion

In [22]:
token_abstract

[['trigramsntags',
  'is',
  'an',
  'efficient',
  'statistical',
  'part-of-speech',
  'tagger',
  'contrary',
  'to',
  'claims',
  'found',
  'elsewhere',
  'in',
  'the',
  'literature',
  'we',
  'argue',
  'that',
  'a',
  'tagger',
  'based',
  'on',
  'markov',
  'models',
  'performs',
  'at',
  'least',
  'as',
  'well',
  'as',
  'other',
  'current',
  'approaches',
  'including',
  'the',
  'maximum',
  'entropy',
  'framework',
  'a',
  'recent',
  'comparison',
  'has',
  'even',
  'shown',
  'that',
  'tnt',
  'performs',
  'significantly',
  'better',
  'for',
  'the',
  'tested',
  'corpora',
  'we',
  'describe',
  'the',
  'basic',
  'model',
  'of',
  'tnt',
  'the',
  'techniques',
  'used',
  'for',
  'smoothing',
  'and',
  'for',
  'handling',
  'unknown',
  'words',
  'furthermore',
  'we',
  'present',
  'evaluations',
  'on',
  'two',
  'corpora'],
 ['this',
  'paper',
  'describes',
  'a',
  'method',
  'for',
  'linear',
  'text',
  'segmentation',
  'whi

# Experimento 1

## Word 2 Vec

sg ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.

In [ ]:
#Skip-Gram
model_sg = Word2Vec(tokens, min_count=1, epochs=100, sg=1)
joblib.dump(model,'model_w2v')

In [23]:
#CBOW
model_cbow = Word2Vec(tokens, min_count=1, epochs=100, sg=0)
joblib.dump(model,'model_w2v')

['model_w2v']

if os.path.exists('model'):
    model = joblib.load('model')
else:
    model = Word2Vec(tokens, min_count=1, epochs=100)
    joblib.dump(model,'model')

In [ ]:
len(model_sg.wv.key_to_index)

In [24]:
len(model_cbow.wv.key_to_index)

50342

## GloVe

https://stackoverflow.com/questions/64261546/how-to-solve-error-microsoft-visual-c-14-0-or-greater-is-required-when-inst

https://towardsdatascience.com/a-comprehensive-python-implementation-of-glove-c94257c2813d

https://github.com/maciejkula/glove-python/issues/121


vocab = glove.build_vocab(sentences)
cooccur = glove.build_cooccur(vocab, sentences, window_size = window_size, min_count = min_count)
id2word = evaluate.make_id2word(vocab)
W = glove.train_glove(vocab, cooccur, vector_size = embedding_size, iterations = 5)
W = evaluate.merge_main_context(W)

from glove import Corpus, Glove

#Creating a corpus object
corpus = Corpus() 

#Training the corpus to generate the co-occurrence matrix which is used in GloVe
corpus.fit(lines, window=10)

glove = Glove(no_components=5, learning_rate=0.05) 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

https://github.com/maciejkula/glove-python

## Fast Text

In [29]:
import fasttext

https://medium.com/@vaibhavshukla182/fasttext-and-how-to-use-it-for-text-analysis-14554cc3b9c

In [30]:
joblib.dump(lista_papers, 'lista_papers.txt')

lista_papers2=''
for i in lista_papers:
    lista_papers2= lista_papers2+ i
    
joblib.dump(lista_papers2, 'string.txt')

['string.txt']

model = fasttext.train_unsupervised('lista_papers.txt', model='skipgram')

len(model.get_words())

In [31]:
model_ft = fasttext.train_unsupervised('string.txt', model='skipgram')

In [32]:
len(model_ft.get_words())

40198

In [33]:
def vetor_frases_fasttext(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model_ft.get_word_vector(j)
            except:
                soma=soma+0
        x.append(soma)
    return x

model2.get_sentence_vector('hello the')

import fasttext


model = fasttext.train_unsupervised('data.txt', model='skipgram')


model = fasttext.train_unsupervised('data.txt', model='cbow')

https://pypi.org/project/fasttext/

## Medida de similaridade

In [34]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<SECTION.*[Ii][nN][tT][rR][oO][dD][Uu][Cc][Tt][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = txt.strip()
        lista.append(txt)
    return txt

def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<[Aa][Bb][sS][tT][Rr][aA][cC][tT]>)(.+?)(</[aA][bB][sS][tT][rR][aA][cC][tT]>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = txt.strip()
        lista.append(txt)
    return txt


def preprocessa_conclusion(corpus: str):
    regIter1 = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos1 = [ t.group(2) for t in regIter1]
    regIter2 = re.finditer('(<SECTION.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos2 = [ t.group(2) for t in regIter2]
    regIter3 = re.finditer('(<SECTION.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Ee][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos3 = [ t.group(2) for t in regIter3]
    textos = textos1
    for i in textos2:
        if i not in textos:
            textos.append(i)
    for i in textos3:
        if i not in textos:
            textos.append(i)
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = txt.strip()
        lista.append(txt)
    return txt

In [ ]:
def funcao_frase_palavra(frases_abstract):
    """input: lista de frases
        output: lista de palavras de cada frase"""
    palavras_frases=[]
    for i in frases_abstract:
        palavras_frases.append(i.split(' '))
    try:
        palavras_frases.remove('')
    except:
        return palavras_frases
    return palavras_frases

In [76]:
def vetor_frases_sg(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model_sg.wv[j]
            except:
                soma=soma+0
        x.append(soma)
    return x

def vetor_frases_cbow(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model_sg.wv[j]
            except:
                soma=soma+0
        x.append(soma)
    return x

In [35]:
def funcao_sim_coseno(a,b):
    if ((numpy.linalg.norm(a)==0) |(numpy.linalg.norm(b)==0)):
        return 0
    else:
        valor_coseno = (numpy.dot(a, b))/(numpy.linalg.norm(a)* numpy.linalg.norm(b))
        return valor_coseno
    
def dicionario_similaridade(vetor_a,vetor_b):
    """Input: vetor correspondente às frases das seções a e b
    Output: 
    """
    dicionario={}
    for i in range(len(vetor_a)):
        for j in range(len(vetor_b)):
            dicionario[i,j]= funcao_sim_coseno(vetor_a[i],vetor_b[j])
    return dicionario

def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

In [36]:
#Definição do Abstract
abstract=[]
for i in papers.values():
    abstract.append(preprocessa_abstract(i))
    
#Definição da Introdução
intro=[]
for i in papers.values():
    intro.append(preprocessa_intro(i))
    
#Definição da Conclusão
conclusion=[]
for i in papers.values():
    conclusion.append(preprocessa_conclusion(i))
    
#Tokens
token_abstract   = []
for texto in abstract:
    token_abstract.append(texto.split())
    
token_intro   = []
for texto in intro:
    token_intro.append(texto.split())
    
token_conclusion   = []
for texto in conclusion:
    token_conclusion.append(texto.split())

tokens = token_abstract+token_intro+token_conclusion

In [37]:
def frases_palavras_vetores(secao, funcao_conversao_vetor):
    frases_intro=[]
    for i in secao:
        frases_intro.append(remove_values_from_list(i.split('.'),''))
        
    frases_intro3=[]
    for i in frases_intro:
        frases_intro2=[]
        for j in i:
            frases_intro2.append(j.strip())
        frases_intro3.append(frases_intro2)

    frases_intro = frases_intro3 
    
    
    palavras_intro=[]
    for i in frases_intro:
        palavras_intro.append(funcao_frase_palavra(i))

    vetor_intro=[]
    for i in palavras_intro:
        vetor_intro.append(funcao_conversao_vetor(i))
        
    return frases_intro,palavras_intro,vetor_intro

In [38]:
def aplicando_dicionario_similaridade(v1,v2):
    x = []
    for i,j in zip(range(len(v1)),range(len(v2))):
        x.append(dicionario_similaridade(v1[i], v2[j]))
        
    top_3_indices = []
    for i in range(len(x)):
        top_3_indices.append(sorted(x[i], key=x[i].get, reverse=True)[:3])
        
    lista_1=[]
    for i in top_3_indices:
        top3=[]
        for j in i:
            top3.append(j[0])
        lista_1.append(list(set(top3)))
        
    return x,top_3_indices,lista_1

In [39]:
def frases_interseccao(lista_1,lista_2):
    indices_objetivo=[]
    for i,j in zip(lista_1,lista_2):
        indices_objetivo.append(list(set.intersection(*map(set,[i,j]))))
    return indices_objetivo

In [40]:
def get_frases_objetivo(indices_objetivo):
    """retorna a frase objetivo a partir dos indices"""
    frase_objetivo=[]
    for idx_f,idx_ind in zip(range(len(frases_abstract)),indices_objetivo):
        lista_one=[]
        for j in idx_ind:
            lista_one.append(frases_abstract[idx_f][j])
            #print(idx_f)
            #print(j)
            #print(frases_abstract[idx_f][j])
        frase_objetivo.append(lista_one)
    return frase_objetivo

In [41]:
#Word2Vec - SkipGram
frases_intro,palavras_intro,vetor_intro                = frases_palavras_vetores(intro, vetor_frases_sg)
frases_abstract,palavras_abstract,vetor_abstract       = frases_palavras_vetores(abstract, vetor_frases_sg)
frases_conclusion,palavras_conclusion,vetor_conclusion = frases_palavras_vetores(conclusion, vetor_frases_sg)

#Abtract e Introdução
sim_ab_intro_w2v,indices_ab_intro_w2v,lista_idx_ab_intro_w2v = aplicando_dicionario_similaridade(vetor_abstract,vetor_intro)

#Abtract e Conclusão
sim_ab_conc_w2v,indices_ab_conc_w2v,lista_idx_ab_c_w2v = aplicando_dicionario_similaridade(vetor_abstract,vetor_conclusion)

#Indices
indices_objetivo_w2v = frases_interseccao(lista_idx_ab_intro_w2v,lista_idx_ab_c_w2v)

#Frases
frase_objetivo_w2v = get_frases_objetivo(indices_objetivo_w2v)

joblib.dump(frase_objetivo_w2v,'frase_objetivo_w2v_sg')

['frase_objetivo_w2v']

In [79]:
#Word2Vec - CBOW
frases_intro,palavras_intro,vetor_intro_cbow                = frases_palavras_vetores(intro, vetor_frases_cbow)
frases_abstract,palavras_abstract,vetor_abstract_cbow       = frases_palavras_vetores(abstract, vetor_frases_cbow)
frases_conclusion,palavras_conclusion,vetor_conclusion_cbow = frases_palavras_vetores(conclusion, vetor_frases_cbow)

#Abtract e Introdução
sim_ab_intro_w2v_cbow ,indices_ab_intro_w2v_cbow ,lista_idx_ab_intro_w2v_cbow  = aplicando_dicionario_similaridade(vetor_abstract_cbow,vetor_intro_cbow)

#Abtract e Conclusão
sim_ab_conc_w2v_cbow,indices_ab_conc_w2v_cbow,lista_idx_ab_c_w2v_cbow = aplicando_dicionario_similaridade(vetor_abstract_cbow,vetor_conclusion_cbow)

#Indices
indices_objetivo_w2v_cbow = frases_interseccao(lista_idx_ab_intro_w2v_cbow,lista_idx_ab_c_w2v_cbow)

#Frases
frase_objetivo_cbow = get_frases_objetivo(indices_objetivo_w2v_cbow)

joblib.dump(frase_objetivo_w2v,'frase_objetivo_w2v_cbow')

['frase_objetivo_w2v_cbow']

In [42]:
#FastText
frases_intro_ft,palavras_intro_ft,vetor_intro_ft                = frases_palavras_vetores(intro, vetor_frases_fasttext)
frases_abstract_ft,palavras_abstract_ft,vetor_abstract_ft       = frases_palavras_vetores(abstract, vetor_frases_fasttext)
frases_conclusion_ft,palavras_conclusion_ft,vetor_conclusion_ft = frases_palavras_vetores(conclusion, vetor_frases_fasttext)


#Abtract e Introdução
sim_ab_intro_ft,indices_ab_intro_ft,lista_idx_ab_intro_ft = aplicando_dicionario_similaridade(vetor_abstract_ft,vetor_intro_ft)

#Abtract e Conclusão
sim_ab_conc_ft,indices_ab_conc_ft,lista_idx_ab_c_ft = aplicando_dicionario_similaridade(vetor_abstract_ft,vetor_conclusion_ft)

#Indices
indices_objetivo_ft = frases_interseccao(lista_idx_ab_intro_ft,lista_idx_ab_c_ft)

#Frases
frase_objetivo_ft = get_frases_objetivo(indices_objetivo_ft)

joblib.dump(frase_objetivo_ft,'frase_objetivo_ft')

['frase_objetivo_ft']

Intersecção entre as frases do abstract que são similares simultâneamente à introdução e a conclusão

`indices_objetivo` índices da frase objetivo

frase_objetivo = []
for i in range(len(frases_abstract)):
    if i in indices_objetivo:
        frase_objetivo.append(frases_abstract[i])

frase_objetivo

Saída para treinamento do modelo de linguagem:
`frase_objetivo`

# Leitura da Sumarização Padrão Ouro

In [43]:
if os.path.exists('lista_padrao_ouro'):
    lista_padrao_ouro = joblib.load('lista_padrao_ouro')
else:
    list_of_files = os.listdir('scisummnet_release1.1__20190413/top1000_complete')

    list_of_files2= []
    for i in list_of_files:
        nome = 'scisummnet_release1.1__20190413/top1000_complete/' + i + '/' + 'summary/' +os.listdir(f'scisummnet_release1.1__20190413/top1000_complete/{i}/summary')[0]
        list_of_files2.append(nome)


    lines = []
    for file in list_of_files2:
        f= open(file,'r', encoding='utf-8' )
        lines.append(f.readlines())
        f.close()

    lista_padrao_ouro=[]
    for j in lines:
        texto=''
        for i in j:
            texto = texto + str(i)
        lista_padrao_ouro.append(texto)

        joblib.dump(lista_padrao_ouro, 'lista_padrao_ouro')

In [44]:
len(lista_padrao_ouro)

1009

In [45]:
#Aplicando indices aos papers
indices           = range(0,len(lista_papers))
padrao_ouro = dict(zip(indices, lista_padrao_ouro))

Filtro dos sumários correspondentes aos papers que possuem as Seções Abstract, Introdução e Conclusão

In [46]:
selecao_padrao_ouro = dict((key,value) for key, value in padrao_ouro.items() if key in  indice_papers)

In [47]:
len(selecao_padrao_ouro)

707

# ROUGE SCORE
Aplicação do `ROUGE` entre as frase identificadas como objetivo e as frases padrão ouro

In [48]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [49]:
selecao_padrao_ouro = list(selecao_padrao_ouro.values())

In [50]:
def pre_processamento_ouro(txt):
    txt = re.sub(r'\n',' ', txt)
    txt = re.sub(r'<.*?>','', txt)
    txt = txt.strip()
    txt = re.sub(' +', ' ', txt)
    #txt = re.sub(' . ', '.', txt).strip()
    #txt = '<sos> ' + txt.strip() + ' <eos>'
    txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
    txt = re.sub('&amp;quot;', '', txt)
    txt = re.sub('  ', ' ', txt)
    txt = re.sub(' +', ' ', txt)
    txt = re.sub('e.g.','',txt)
    txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
    txt = re.sub(r'\d', '', txt) #removendo demais numeros
    txt = txt.lower()
    txt = txt.strip()
    return txt

In [51]:
def pre_processamento_ouro2(txt):
    txt = txt.strip()
    return txt

In [52]:
def calculo_rouge(frase_objetivo,lista_frases_padrao_ouro):
    rouge_total=[]
    for i,j in zip(frase_objetivo,lista_frases_padrao_ouro):
        rouge_1={}
        for h in range(len(i)):
            for z in range(len(j)):
                rouge_1[(h,z)] = (scorer.score(i[h],j[z])['rouge1'].fmeasure)
        rouge_total.append(rouge_1)
    return rouge_total

In [53]:
def get_max(dic, coord, val):
    return max(filter(lambda item: item[0][coord] == val, dic.items()), key=lambda x: x[1])

In [54]:
def max_rouge(rouge_total):
    """pega o máximo rouge score para cada frase selecionada como pertencente ao objetivo, em comparação com cada frase
    do padrão ouro"""
    maximo_rouge_por_paper =[]
    for h in rouge_total:
        x=[]
        for i in set([i[0] for i in h]):
            x.append(get_max(h,0,i))
        maximo_rouge_por_paper.append(x)
    return maximo_rouge_por_paper

In [55]:
lista_frases_padrao_ouro=[]
for i in selecao_padrao_ouro:
    texto = pre_processamento_ouro(i).strip().split('.')
    texto = remove_values_from_list(texto,'')
    lista_frases_padrao_ouro.append(texto)

In [56]:
x=[]
for i in lista_frases_padrao_ouro:
    x.append(len(i))
media_ouro=np.mean(x)
media_ouro

6.5954738330975955

In [57]:
rouge_ft = calculo_rouge(frase_objetivo_ft,lista_frases_padrao_ouro)
rouge_w2v = calculo_rouge(frase_objetivo_w2v,lista_frases_padrao_ouro)
rouge_cbow  = calculo_rouge(frase_objetivo_cbow,lista_frases_padrao_ouro)

In [59]:
maximo_rouge_por_paper_ft = max_rouge(rouge_ft)
maximo_rouge_por_paper_w2v = max_rouge(rouge_w2v)
maximo_rouge_por_paper_cbow = max_rouge(rouge_cbow)

In [84]:
#Conferindo
rouge_ft==rouge_cbow

False

#conferindo
rouge_1={}
for i in range(len(frase_objetivo[0])):
    for j in range(len(frases_padra_ouro)):
        rouge_1[(i,j)] = (scorer.score(frase_objetivo[0][i],frases_padra_ouro[j])['rouge1'].fmeasure)
rouge_1

In [62]:
def rouge_precisao_revocacao(maximo_rouge_por_paper):
    precisao=[]
    for i in maximo_rouge_por_paper:
        soma=0
        for j in i:
            soma = soma + j[1]
        #print(len(i))
        try:
            precisao.append(soma/len(i))
        except:
            precisao.append(0)

    revocacao=[]
    for i,h in zip(maximo_rouge_por_paper,lista_frases_padrao_ouro):
        soma=0
        for j in i:
            soma = soma + j[1]
        #print(len(i))
        try:
            revocacao.append(soma/len(h))
        except:
            revocacao.append(0)
    return precisao, revocacao

In [63]:
precisao_ft, revocacao_ft = rouge_precisao_revocacao(maximo_rouge_por_paper_ft)

In [64]:
precisao_w2v, revocacao_w2v = rouge_precisao_revocacao(maximo_rouge_por_paper_w2v)

In [85]:
precisao_cbow, revocacao_cbow = rouge_precisao_revocacao(maximo_rouge_por_paper_cbow)

dicionario_revocacao={}
for i,j in zip(revocacao,indice_papers):
    dicionario_revocacao[j] = i
dicionario_revocacao

dicionario_precisao={}
for i,j in zip(precisao,indice_papers):
    dicionario_precisao[j] = i
dicionario_precisao

# Conclusão do Experimento 1

In [65]:
def analise_precisao(precisao):
    x=[]
    qtd_zero=[]
    for i in precisao:
        if i>=0.8:
            x.append(i)
        if i==0:
            qtd_zero.append(i)
    print('qtd de papers com precisao>=0.8:',len(x))
    print('% do total:',len(x)/len(lista_frases_padrao_ouro))
    print('qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções:',len(qtd_zero))
    print('% do total:',len(qtd_zero)/len(lista_frases_padrao_ouro))
    print('media precisao:', np.mean(precisao))
    return len(x), len(qtd_zero),np.mean(precisao)

In [86]:
qtd_precisao_08_cbow, qtd_zero_cbow, media_cbow = analise_precisao(precisao_cbow)

qtd de papers com precisao>=0.8: 357
% do total: 0.504950495049505
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 28
% do total: 0.039603960396039604
media precisao: 0.7476895255271888


In [66]:
qtd_precisao_08_w2v, qtd_zero_w2v, media_w2v = analise_precisao(precisao_w2v)

qtd de papers com precisao>=0.8: 532
% do total: 0.7524752475247525
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 111
% do total: 0.157001414427157
media precisao: 0.7671501147238049


In [67]:
qtd_precisao_08_ft, qtd_zero_ft,  media_ft = analise_precisao(precisao_ft)

qtd de papers com precisao>=0.8: 581
% do total: 0.8217821782178217
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 67
% do total: 0.09476661951909476
media precisao: 0.8334893094200506


In [101]:
precisao_mean =  {'media_ft':media_ft,
                  'media_w2v':media_w2v,
                  'media_cbow':media_cbow}
max(precisao_mean, key=precisao_mean.get)

'media_ft'

In [103]:
qtd_papers_hipotese_invalida = {'qtd_zero_w2v':qtd_zero_w2v,
                              'qtd_zero_ft':qtd_zero_ft,
                              'qtd_zero_cbow':qtd_zero_cbow}
min(qtd_papers_hipotese_invalida, key=qtd_papers_hipotese_invalida.get)

'qtd_zero_cbow'

In [107]:
qtd_precisao_alta = {'qtd_precisao_08_w2v':qtd_precisao_08_w2v,
                     'qtd_precisao_08_ft':qtd_precisao_08_ft,
                     'qtd_precisao_08_cbow':qtd_precisao_08_cbow}
max(qtd_precisao_alta, key=qtd_precisao_alta.get)

'qtd_precisao_08_ft'

In [71]:
def analise_revocacao(revocacao):
    x=[]
    qtd_zero=[]
    for i in revocacao:
        if i>=(1/media_ouro): #valor de 1/média qtd frases padrao ouro
            x.append(i)
        if i==0:
            qtd_zero.append(i)
    print('qtd de papers com revocação>=(1/media_ouro):',len(x))
    print('% do total:',len(x)/len(lista_frases_padrao_ouro))
    print('qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções:',len(qtd_zero))
    print('% do total:',len(qtd_zero)/len(lista_frases_padrao_ouro))
    return len(x), np.mean(revocacao)

In [111]:
qtd_revocacao_w2v, media_revocacao_w2v      = analise_revocacao(revocacao_w2v)
qtd_revocacao_ft,  media_revocacao_ft       = analise_revocacao(revocacao_ft)
qtd_revocacao_cbow,  media_revocacao_cbow   = analise_revocacao(revocacao_cbow)

qtd de papers com revocação>=(1/media_ouro): 414
% do total: 0.5855728429985856
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 111
% do total: 0.157001414427157
qtd de papers com revocação>=(1/media_ouro): 455
% do total: 0.6435643564356436
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 67
% do total: 0.09476661951909476
qtd de papers com revocação>=(1/media_ouro): 200
% do total: 0.2828854314002829
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 28
% do total: 0.039603960396039604


In [120]:
qtd_revocacao_alta = {'qtd_revocacao_w2v':qtd_precisao_08_w2v,
                     'qtd_revocacao_ft':qtd_precisao_08_ft,
                     'qtd_revocacao_cbow':qtd_precisao_08_cbow}
max(qtd_revocacao_alta, key=qtd_revocacao_alta.get)

'qtd_revocacao_ft'

In [122]:
media_revocacao   = {'media_revocacao_w2v':media_revocacao_w2v,
                     'media_revocacao_ft':media_revocacao_ft,
                     'media_revocacao_cbow':media_revocacao_cbow}
max(media_revocacao, key=media_revocacao.get)

'media_revocacao_ft'

In [73]:
max([qtd_revocacao_w2v,qtd_revocacao_ft])

637

In [74]:
max([media_revocacao_w2v,media_revocacao_ft])

0.8334893094200506

# Modelo de Linguagem

limpar o texto usado para treinar o fasttext
baixar biblioteca glove
implementar skip-gram e cbow
treinar o modelo de linguagem

https://pypi.org/project/glove/